# 「Pythonでスラスラわかる ベイズ推論「超」入門」をNumPyroで書き直す-目次

## はじめに
[Pythonでスラスラわかる ベイズ推論「超」入門(赤石 雅典 (著), 須山 敦志 (監修))](https://www.yodobashi.com/product/100000009003770765/)はとても分かりやすい良書です。\
この記事は当該書籍のベイズ推論のライブラリを[PyMC](https://www.pymc.io/welcome.html)から[NumPyro](https://num.pyro.ai/en/stable/)に書き換えたコード(Zenn記事)への**リンク集**です。

異なるライブラリを使って同じ結果を得る過程は良い学びとなります。\
それぞれのライブラリの共通項を理解することでその裏にある原理への理解も深まります。\
これらの記事が皆さんの参考になれば幸いです。

## PyMCとNumPyroの比較
それぞれのチュートリアルを一通り触ってみての所感です。\
最初は癖の少ないPyMCで学ぶのが良いと思います。\
速度が必要になってからNumPyroに移るのがよいでしょう。

項目 | PyMC | NumPyro |
---- | ---- | ---- |
特徴量と目的変数のデータ型 | ```numpy.ndarray```, ```pandas.DataFrame```を使用できる。慣れている人が多い。 | ```jax.numpy.array```を使う。 |
確率モデルの作り方 | モデルのインスタンスを生成した後に```with```構文内でモデルの詳細を実装する。 | モデル名の関数の中でモデルの詳細を実装する。こちらの方が好み。 |
確率モデルのプロット | 説明変数も可視化できる。わかりやすい。 | プロットがすっきりする。 |
確率モデルのプロットのコードの書きやすさ | 簡単。書籍の通りでOK。 | 面倒くさい。ユーティリティ関数を作るべき。(本記事にコードを掲載しました) |
サンプリングの実行速度 | 遅い。他のバックエンドを使用しても遅い。 | **速い**。もう戻れない。 |
サンプリングのコードの書きやすさ | 簡単。書籍の通りでOK。 | 面倒くさい。ユーティリティ関数を作るべき。(本記事にコードを掲載しました) |
サンプリング後の検証 | ArviZのコードを使用するので同じコードを使用する。 | 左に同じ |

## ライブラリのバージョン
活発に更新されているライブラリでは破壊的変更が行われることがあります。\
読者が同じコードを実行できるように、本記事とリンク先で使用するライブラリのバージョンを列挙します。\
First releasedから一年たつのでそろそろ```Python 3.13```に引き上げても良いだろうと判断しました。

In [ ]:
# Pythonのバージョン
import sys
print("Miniforge", sys.version)

# DataFrame, Numerical computation
#import polars as pl
#print("Polars", pl.__version__)
import pandas as pd
print("Pandas", pd.__version__)
import numpy as np
print("NumPy",np.__version__)
import jax
print("Jax", jax.__version__)

# ベイズ推定
import numpyro
print("NumPyro", numpyro.__version__)

# plot
import matplotlib
print("Matplotlib", matplotlib.__version__)
%matplotlib inline
import seaborn as sns
print("seaborn", sns.__version__)
import arviz as az
print("ArviZ", az.__version__)

# plotの設定
import json
print("JSON", json.__version__)

# plotで日本語フォントを使用する。
import japanize_matplotlib
#print(japanize_matplotlib.__version__)

## フォルダ構造とユーティリティ
